<h1>NMF piano transcription</h1>
In this tutorial, we look at the use of NMF to decompose a signal into a "notes-based" representation. We compare two versions: first, with random initialisation, and second, with harmonic templates for each pitch in the audio.

<h2>NMF main method</h2>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
%matplotlib inline

@jit(nopython=True)
def NMF(V, R, thresh=0.001, L=1000, W=None, H=None, norm=False, report=False):
    """NMF algorithm with Euclidean distance
    
    Based on FMP Notebook: C8/C8S3_NMFbasic.ipynb
    
    Args: 
        V: Input spectrogram: Nonnegative matrix of size K x N
        R: Rank parameter: Number of templates in W
        thresh: Threshold used as stop criterion
        L: Maximal number of iterations
        W: Nonnegative matrix of size K x R used for initialization
        H: Nonnegative matrix of size R x N used for initialization
        norm (bool): Applies max-normalization of columns of final W
        report (bool): Reports errors during runtime
    
    Returns: 
        W: Dictionary of spectral vectors: Nonnegative matrix of size K x R
        H: Activation matrix: Nonnegative matrix of size R x N
        V_approx: Nonnegative matrix W*H of size K x N
        V_approx_err: Error between V and V_approx
        H_W_error: History of differences between subsequent H and W matrices
    """ 
    
    # Initialise variables
    K = V.shape[0]
    N = V.shape[1]
    if W is None:
        W = np.random.rand(K,R)
    if H is None:
        H = np.random.rand(R,N)
    V = V.astype(np.float64)
    W = W.astype(np.float64)
    H = H.astype(np.float64) 
    H_W_error = np.zeros((2,L))
    ell = 1
    below_thresh = False
    eps_machine = np.finfo(np.float32).eps  # small number to avoid divide by zero errors
    
    # Main loop
    while not below_thresh and ell <= L:
        H_ell = H
        W_ell = W
        # The multiplicative update equations
        H = 
        W = 
        # Calculate change (not sure why Meinard called these "error"s)
        H_error = np.linalg.norm(H - H_ell, ord=2)
        W_error = np.linalg.norm(W - W_ell, ord=2)
        H_W_error[:, ell-1] = [H_error, W_error]
        # Print results
        if report:
            print('Iteration: ',ell,', H_error: ',H_error,', W_error: ',W_error)            
        # Update parameters
        if H_error < thresh and W_error < thresh:
            below_thresh = True
            H_W_error = H_W_error[:,0:ell]   # cut matrix to required length 
        ell += 1
    if norm:        # Rescale W to range [0,1] for each vector; adjust H to keep WH unchanged
        for r in range(R):
            v_max = np.max(W[:,r])
            if v_max > 0:
                W[:,r] = W[:,r] / v_max
                H[r,:] = H[r,:] * v_max         
    V_approx = W.dot(H)
    V_approx_err = np.linalg.norm(V-V_approx, ord=2)   # This one is the approximation error
    return W, H, V_approx, V_approx_err, H_W_error

<h2>Helper methods (display)</h2>

In [ ]:
import matplotlib.pyplot as plt

def plot_NMF(V, W, H, V_approx, error, figsize=(10,2), aspect='auto', wr=[1, 0.5, 1, 1]): 
    fig, ax = plt.subplots(1, 4, gridspec_kw={'width_ratios': wr},
                           figsize=figsize)    
    cmap = 'gray_r'
    im = ax[0].imshow(V, aspect=aspect, origin='lower', cmap=cmap, clim=[0, np.max(V)])
    ax[0].set_title(r'$V$')
    plt.sca(ax[0])
    plt.colorbar(im)   
    im = ax[1].imshow(W, aspect=aspect, origin='lower', cmap=cmap, clim=[0, np.max(W)])
    ax[1].set_title(r'$W$')
    plt.sca(ax[1])
    plt.colorbar(im)
    im = ax[2].imshow(H, aspect=aspect, origin='lower', cmap=cmap, clim=[0, np.max(H)])
    ax[2].set_title(r'$H$')
    plt.sca(ax[2])    
    plt.colorbar(im)
    im = ax[3].imshow(V_approx, aspect=aspect, origin='lower', cmap=cmap, clim=[0, np.max(V_approx)])
    ax[3].set_title(r'$WH$ (error = %0.2f)'%error)
    plt.sca(ax[3])    
    plt.colorbar(im)
    plt.tight_layout() 
    plt.show()
    
def plot_NMF_factors(V, W, H, WH, error, Fs, N_fft, H_fft, freq_max, label_pitch=None, 
                     title_V='V', title_W='W', title_H='H', title_WH='WH', figsize=(12,2)):
    """Plots the factors of an NMF-based spectral decomposition

    FMP Notebook: C8/C8S3_NMFSpecFac.ipynb
    """
    R = W.shape[1]
    N = H.shape[1]
    cmap = 'gray_r'
    dur_sec = (N-1) * H_fft / Fs
    if label_pitch is None:
        label_pitch = np.arange(R)

    plt.figure(figsize=figsize)

    plt.subplot(141)
    plt.imshow(V, cmap=cmap, origin='lower', aspect='auto', extent=[0, dur_sec, 0, freq_max])
    plt.ylim([0, freq_max]);
    plt.colorbar()
    plt.xlabel('Time (seconds)')
    plt.ylabel('Frequency (Hz)')
    plt.title(title_V)

    plt.subplot(142)
    plt.imshow(W, cmap=cmap, origin='lower', aspect='auto', extent=[0, R, 0, freq_max])
    # correction: freq_max was Fs/2 in imshow call above; also in 3rd subplot
    plt.ylim([0, freq_max])
    plt.colorbar()
    plt.xticks(np.arange(R) + 0.5, label_pitch)
    plt.xlabel('Pitch')
    plt.ylabel('Frequency (Hz)')
    plt.title(title_W)

    plt.subplot(143)
    plt.imshow(H, cmap=cmap, origin='lower', aspect='auto', extent=[0, dur_sec, 0, R])
    plt.colorbar()
    plt.yticks(np.arange(R) + 0.5, label_pitch)
    plt.xlabel('Time (seconds)')
    plt.ylabel('Pitch')
    plt.title(title_H)

    plt.subplot(144)
    plt.imshow(WH, cmap=cmap, origin='lower', aspect='auto', extent=[0, dur_sec, 0, freq_max])
    plt.ylim([0, freq_max]);
    plt.colorbar()
    plt.xlabel('Time (seconds)')
    plt.ylabel('Frequency (Hz)')
    plt.title(title_WH + ' (Error = {:4.2f})'.format(error))

    plt.tight_layout()
    plt.show()


<h2>Run an example of NMF with random initialisation and show results</h2>
Three examples are given here, a one-octave scale (monophonic), two bars of a Chopin Prelude (Op.28 No.4), and one minute of a Chopin Etude (Op.10 No.3).

In [ ]:
import librosa
Fs = 22050

Example = 2   # CHOOSE 1,2 or 3 here

if Example == 1:   # 8-note scale, monophonic
    fname = 'FMP_C3_F08_C-major-scale.wav'  # in ../../resources/python/FMP_0.1.1/data/C3/
    pitch_set = np.asarray([60,62,64,65,67,69,71,72])
elif Example == 2: # two bars of a Chopin Prelude (Op.28 No.4), also only 8 distinct pitches, but polyphonic
    fname = 'FMP_C8_Audio_Chopin_Op028-04_SMD_beginning.wav'
    # or the whole piece:  '../../resources/python/FMP_0.1.1/data/C8/SyncScoreAudio/FMP_C8_Audio_Chopin_Op028-04_SMD.csv'
#    import sys
#    sys.path.append('../../resources/python/FMP_0.1.1/')
#    import LibFMP.C1
#    fn_ann = 'FMP_C8_Audio_Chopin_Op028-04_SMD_beginning.csv'
#    annotation = LibFMP.C1.pianoroll.csv_to_list(fn_ann)
#    pitch_set = pitch_from_annotation(annotation)
    pitch_set = np.asarray([54,55,57,59,63,64,71,72])  # hard-coded to remove dependencies
elif Example == 3:  # ~1 minute excerpt of Chopin Etude Op.10 No.3
    fname = '01etd_bo.wav'
    pitch_set = np.asarray(range(35,81))
    # Actual pitches used: [35,40,44,45,47,48,49,51,52,54,56,57,58,59,61,62,63,64,66,68,69,70,71,72,73,75,76,78,80]

# Load audio and compute spectrogram
x, Fs = librosa.load(fname, sr=Fs)
Nfft, Hop = 2048, 1024
X = librosa.stft(x, n_fft=Nfft, hop_length=Hop, win_length=Nfft, window='hann', center=True, pad_mode='constant')
V = np.log(1 + 10 * np.abs(X))
maxFreqIndex = 120   # Limit to 1292Hz, f0 of Eb6
V = V[0:maxFreqIndex, :]

# Initialise variables
K = V.shape[0]
N = V.shape[1]
R = len(pitch_set)      # expected or maximum number of unique pitches
thresh = 0.0001
L = 200
W_init =  np.random.rand(K,R)  # Computed here so we can plot it and calculate error; otherwise unnecessary
W_init = W_init/np.max(W_init)
H_init = np.random.rand(R,N)  

# Show initial (random) result
V_approx = W_init.dot(H_init)
error = np.linalg.norm(V-V_approx, ord=2)
print('Matrix V and randomly initialized matrices W and H')
plot_NMF(V, W_init, H_init, V_approx, error, figsize=(12,2), wr=[1, 1, 1, 1])

# Run NMF algorithm and show final result
W, H, V_approx, V_approx_err, H_W_error = NMF(V, R, thresh=thresh, 
                                              L=L, W=W_init, H=H_init, norm=1, report=0)
print('Matrix V and matrices W and H after training')
plot_NMF(V, W, H, V_approx, V_approx_err, figsize=(12,2), wr=[1, 1, 1, 1])

In [ ]:
# Postprocessing to order elements by frequency of strongest partial
M = np.argmax(W, axis=0)     # For each column of W, gives the position of the maximum
indices = M.argsort()        # Get index values to sort columns of W (and rows of H)
W2 = W[:,indices]            # Re-order columns of W
H2 = H[indices,:]            # Re-order rows of H in the same way, so that W2.H2 == W.H
V2approx = W2.dot(H2)        # Check that this worked (V2approx should be the same as V_approx)
error2 = np.linalg.norm(V-V2approx, ord=2)  # Likewise the error should not be changed
print('Matrix V and matrices W and H after training and sorting templates')
plot_NMF(V, W2, H2, V2approx, error2, figsize=(12,2), wr=[1, 1, 1, 1])    # Shows results

<h2>Methods for initialising W with harmonic templates</h2>

In [ ]:
def pitch_from_annotation(annotation):
    """Extract set of occurring pitches from annotation
    
    FMP Notebook: C8/C8S3_NMFSpecFac.ipynb
    
    Args: 
        annotation: Annotation data
    
    Returns: 
        pitch_set: Set of occurring pitches
    """    
    pitch_all = np.array([c[2] for c in annotation])
    pitch_set = np.unique(pitch_all)
    return pitch_set


def template_pitch(K, pitch, freq_res, tol_pitch=0.05):
    """Defines spectral template for a given pitch
    
    FMP Notebook: C8/C8S3_NMFSpecFac.ipynb
    
    Args: 
        K: Number of frequency points
        pitch: Fundamental pitch
        freq_res: Frequency resolution
        tol_pitch: Relative frequency tolerance for the harmonics
    
    Returns: 
        template: Nonnegative vector of size K
    """    
    max_freq = K * freq_res
    pitch_freq = 2**((pitch - 69) / 12) * 440
    max_order = int(np.ceil(max_freq / ((1 - tol_pitch) * pitch_freq)))
    template = np.zeros(K)
    for m in range(1, max_order + 1):
        min_idx = max(0, int((1- tol_pitch) * m * pitch_freq / freq_res))
        max_idx = min(K-1, int((1 + tol_pitch) * m * pitch_freq / freq_res))
        template[min_idx:max_idx+1] = 1/m
    return template


def init_NMF_template_pitch(K, pitch_set, freq_res, tol_pitch=0.05):
    """Initializes template matrix for a given set of pitches
    
    FMP Notebook: C8/C8S3_NMFSpecFac.ipynb
    
    Args: 
        K: Number of frequency points
        pitch_set: Set of fundamental pitches
        freq_res: Frequency resolution
        tol_pitch: Relative frequency tolerance for the harmonics
    
    Returns: 
        W: Nonnegative matrix of size K x R with R = len(pitch_set)
    """      
    R = len(pitch_set)
    W = np.zeros((K,R))
    for r in range(R):
        W[:,r] = template_pitch(K, pitch_set[r], freq_res, tol_pitch=tol_pitch)    
    return W  

In [ ]:
# Initialise W with harmonic series for each pitch
K = V.shape[0]
N = V.shape[1]
R = pitch_set.shape[0]
freq_res = Fs / Nfft
freq_max = K * freq_res
W_init = init_NMF_template_pitch(K, pitch_set, freq_res, tol_pitch=0.05)
H_init = np.random.rand(R,N)
V_approx = W_init.dot(H_init)
V_approx_err = np.linalg.norm(V-V_approx, ord=2) 
plot_NMF_factors(V, W_init, H_init, W_init.dot(H_init), V_approx_err, Fs, Nfft, Hop, freq_max, label_pitch=pitch_set)
W, H, V_approx, V_approx_err, H_W_error = NMF(V, R, W=W_init, H=H_init, L=100, norm=True)
plot_NMF_factors(V, W, H, W.dot(H), V_approx_err, Fs, Nfft, Hop, freq_max, label_pitch=pitch_set)